# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
import geoviews
import pyproj

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,6.34,93,37,2.61,NZ,1721314759
1,1,nova sintra,14.8667,-24.7167,24.42,74,14,4.30,CV,1721314760
2,2,margaret river,-33.9500,115.0667,10.93,62,99,14.03,AU,1721314761
3,3,barranquilla,10.9639,-74.7964,27.88,83,20,2.06,CO,1721314544
4,4,adamstown,-25.0660,-130.1015,22.18,86,6,9.10,PN,1721314763


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [42]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"]

# Display the map
# https://hvplot.holoviz.org/reference/geopandas/points.html
map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color="City",
    coastline=True, 
    size="Humidity"
)

# Display the map
map_1

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Lng,Lat]   (City,Humidity)
   .Coastline.I :Feature   [Longitude,Latitude]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower or equal than 35 degrees but higher or equal than 25
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= 35) & (city_data_df["Max Temp"] >= 25)]

#Wind speed less or equal than 3 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] <= 3)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,barranquilla,10.9639,-74.7964,27.88,83,20,2.06,CO,1721314544
5,5,manaure,11.7751,-72.4445,29.82,63,100,2.53,CO,1721314764
20,20,chardara,41.2547,67.9692,29.94,32,24,1.60,KZ,1721314782
23,23,kathu,7.9178,98.3332,29.83,87,98,2.81,TH,1721314785
24,24,archangelos,36.2149,28.1149,29.99,38,0,2.82,GR,1721314786
...,...,...,...,...,...,...,...,...,...,...
545,545,vilhena,-12.7406,-60.1458,28.86,28,0,2.72,BR,1721315436
558,558,san luis de la loma,17.2706,-100.8939,30.23,54,22,0.37,MX,1721315452
561,561,entebbe,0.0644,32.4469,27.18,57,20,2.57,UG,1721315456
567,567,vernon,50.2581,-119.2691,25.72,49,84,0.61,CA,1721315385


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = pd.DataFrame({"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity
                        })

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,barranquilla,CO,10.9639,-74.7964,83,
5,manaure,CO,11.7751,-72.4445,63,
20,chardara,KZ,41.2547,67.9692,32,
23,kathu,TH,7.9178,98.3332,87,
24,archangelos,GR,36.2149,28.1149,38,
...,...,...,...,...,...,...
545,vilhena,BR,-12.7406,-60.1458,28,
558,san luis de la loma,MX,17.2706,-100.8939,54,
561,entebbe,UG,0.0644,32.4469,57,
567,vernon,CA,50.2581,-119.2691,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
barranquilla - nearest hotel: Hotel 56 Avenue
manaure - nearest hotel: hotel palaima
chardara - nearest hotel: Бақдаулет
kathu - nearest hotel: B&B NAMO PHUKET
archangelos - nearest hotel: Semina
beinamar - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
newport - nearest hotel: Queens Hotel
bethel - nearest hotel: Hampton Inn Danbury
remire-montjoly - nearest hotel: Complexe Belova
tura - nearest hotel: Hotel Polo Orchid
sofifi - nearest hotel: No hotel found
ebaye - nearest hotel: No hotel found
weiser - nearest hotel: State Street Motel
porto de moz - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
balotra - nearest hotel: No hotel found
rudsar - nearest hotel: هتل بهار
pahrump - nearest hotel: Holiday Inn Express & Suites
fethiye - nearest hotel: No hotel found
sao joao da barra - nearest hotel: Pousada Mediterrâneo
olathe - nearest hotel: Best Western Plus Ola

,City,Country,Lat,Lng,Humidity,Hotel Name
3,barranquilla,CO,10.9639,-74.7964,83,Hotel 56 Avenue
5,manaure,CO,11.7751,-72.4445,63,hotel palaima
20,chardara,KZ,41.2547,67.9692,32,Бақдаулет
23,kathu,TH,7.9178,98.3332,87,B&B NAMO PHUKET
24,archangelos,GR,36.2149,28.1149,38,Semina
...,...,...,...,...,...,...
545,vilhena,BR,-12.7406,-60.1458,28,No hotel found
558,san luis de la loma,MX,17.2706,-100.8939,54,No hotel found
561,entebbe,UG,0.0644,32.4469,57,Airport View Hotel
567,vernon,CA,50.2581,-119.2691,49,Okanagan Royal Park Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color="City",
    coastline=True, 
    hover_cols=["Hotel Name", "Country"],
    size="Humidity"
)

# Display the map
map_2

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)
   .Coastline.I :Feature   [Longitude,Latitude]